In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

class FMPDataPuller:
    def __init__(self, api_key, tickers):
        self.api_key = api_key
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.base_url = "https://financialmodelingprep.com/api/v3"
        
    def _make_request(self, endpoint, params=None):
        """Make API request with error handling"""
        if params is None:
            params = {}
        params['apikey'] = self.api_key
        
        try:
            response = requests.get(f"{self.base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return None
    
    def get_current_fundamentals(self):
        """Get current fundamental data for all tickers"""
        print("Fetching fundamental data from FMP...")
        fundamental_data = {}
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                # Get company profile for basic info
                profile_data = self._make_request(f"profile/{ticker}")
                
                # Get key metrics
                metrics_data = self._make_request(f"key-metrics-ttm/{ticker}")
                
                # Get ratios
                ratios_data = self._make_request(f"ratios-ttm/{ticker}")
                
                # Get latest income statement (annual)
                income_data = self._make_request(f"income-statement/{ticker}", params={'limit': 1})
                
                # Get latest balance sheet (annual)  
                balance_data = self._make_request(f"balance-sheet-statement/{ticker}", params={'limit': 1})
                
                # Get latest cash flow (annual)
                cashflow_data = self._make_request(f"cash-flow-statement/{ticker}", params={'limit': 1})
                
                # Extract the data we need
                profile = profile_data[0] if profile_data else {}
                metrics = metrics_data[0] if metrics_data else {}
                ratios = ratios_data[0] if ratios_data else {}
                income = income_data[0] if income_data else {}
                balance = balance_data[0] if balance_data else {}
                cashflow = cashflow_data[0] if cashflow_data else {}
                
                fundamental_data[ticker] = {
                    'total_revenue': income.get('revenue'),
                    'operating_income': income.get('operatingIncome'),
                    'ebitda': income.get('ebitda'),
                    'net_income': income.get('netIncome'),
                    'total_debt': balance.get('totalDebt'),
                    'cash_and_equivalents': balance.get('cashAndCashEquivalents'),
                    'free_cash_flow': cashflow.get('freeCashFlow'),
                    'pe_ratio': ratios.get('priceEarningsRatio'),
                    'sector': profile.get('sector'),
                    'industry': profile.get('industry'),
                    'market_cap': profile.get('mktCap'),
                    'employee_count': profile.get('fullTimeEmployees'),
                    'last_updated': datetime.now()
                }
                
            except Exception as e:
                print(f"  Error fetching data for {ticker}: {e}")
                fundamental_data[ticker] = {
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.2)  # Rate limiting
            
        return pd.DataFrame(fundamental_data).T
    
    def get_quarterly_revenue(self, periods=8):
        """Get quarterly revenue data for the past N quarters"""
        print(f"Fetching quarterly revenue data for {periods} quarters from FMP...")
        quarterly_data = {}
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                # Get quarterly income statements (limit to periods + 2 for safety)
                quarterly_income = self._make_request(
                    f"income-statement/{ticker}", 
                    params={'period': 'quarter', 'limit': periods + 2}
                )
                
                if quarterly_income:
                    quarterly_revenue = []
                    
                    # Process the quarterly data
                    for quarter in quarterly_income[:periods]:
                        if quarter.get('revenue'):
                            # Parse the date and create quarter string
                            date_str = quarter.get('date', '')
                            if date_str:
                                try:
                                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                                    quarter_num = ((date_obj.month - 1) // 3) + 1
                                    quarter_str = f"{date_obj.year}-Q{quarter_num}"
                                except:
                                    quarter_str = date_str
                            else:
                                quarter_str = quarter.get('calendarYear', 'Unknown')
                            
                            quarterly_revenue.append({
                                'quarter': quarter_str,
                                'date': date_str,
                                'revenue': quarter.get('revenue'),
                                'period': quarter.get('period', 'Q')
                            })
                    
                    quarterly_data[ticker] = {
                        'quarterly_revenue': quarterly_revenue,
                        'quarters_available': len(quarterly_revenue),
                        'last_updated': datetime.now()
                    }
                    
                    print(f"  Found {len(quarterly_revenue)} quarters of data")
                    
                else:
                    print(f"  No quarterly data available for {ticker}")
                    quarterly_data[ticker] = {
                        'quarterly_revenue': [],
                        'error': 'No quarterly income statement data',
                        'last_updated': datetime.now()
                    }
                    
            except Exception as e:
                print(f"  Error fetching quarterly data for {ticker}: {e}")
                quarterly_data[ticker] = {
                    'quarterly_revenue': [],
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.2)  # Rate limiting
            
        return quarterly_data
    
    def get_all_quarterly_metrics(self, periods=8):
        """Get comprehensive quarterly data (revenue, operating income, ebitda, net income)"""
        print(f"Fetching comprehensive quarterly data for {periods} quarters...")
        quarterly_data = {}
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                # Get quarterly income statements
                quarterly_income = self._make_request(
                    f"income-statement/{ticker}", 
                    params={'period': 'quarter', 'limit': periods + 2}
                )
                
                if quarterly_income:
                    quarterly_metrics = []
                    
                    for quarter in quarterly_income[:periods]:
                        # Parse the date
                        date_str = quarter.get('date', '')
                        if date_str:
                            try:
                                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                                quarter_num = ((date_obj.month - 1) // 3) + 1
                                quarter_str = f"{date_obj.year}-Q{quarter_num}"
                            except:
                                quarter_str = date_str
                        else:
                            quarter_str = f"{quarter.get('calendarYear', 'Unknown')}-Q{quarter.get('period', '?')}"
                        
                        quarterly_metrics.append({
                            'quarter': quarter_str,
                            'date': date_str,
                            'revenue': quarter.get('revenue'),
                            'operating_income': quarter.get('operatingIncome'),
                            'ebitda': quarter.get('ebitda'),
                            'net_income': quarter.get('netIncome'),
                            'eps': quarter.get('eps'),
                            'gross_profit': quarter.get('grossProfit')
                        })
                    
                    quarterly_data[ticker] = {
                        'quarterly_metrics': quarterly_metrics,
                        'quarters_available': len(quarterly_metrics),
                        'last_updated': datetime.now()
                    }
                    
                else:
                    quarterly_data[ticker] = {
                        'quarterly_metrics': [],
                        'error': 'No quarterly data available',
                        'last_updated': datetime.now()
                    }
                    
            except Exception as e:
                print(f"  Error: {e}")
                quarterly_data[ticker] = {
                    'quarterly_metrics': [],
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.2)
            
        return quarterly_data

# Example usage and testing
if __name__ == "__main__":
    # You'll need to get your API key from https://financialmodelingprep.com/
    # API_KEY = "your_api_key_here"
    
    # Example with test tickers
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
    
    # Initialize (you need to provide your API key)
    # fmp = FMPDataPuller(API_KEY, tickers)
    
    # Test current fundamentals
    # fundamentals = fmp.get_current_fundamentals()
    # print("\nFundamental Data:")
    # print(fundamentals[['total_revenue', 'ebitda', 'net_income', 'market_cap']].head())
    
    # Test quarterly revenue (8 quarters)
    # quarterly_revenue = fmp.get_quarterly_revenue(periods=8)
    # print("\nQuarterly Revenue Data:")
    # for ticker, data in quarterly_revenue.items():
    #     if 'error' not in data:
    #         print(f"\n{ticker}: {data['quarters_available']} quarters")
    #         for q in data['quarterly_revenue'][:3]:
    #             print(f"  {q['quarter']}: ${q['revenue']/1e9:.2f}B")
    
    print("FMP Data Puller ready - add your API key to test!")

FMP Data Puller ready - add your API key to test!


In [8]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

class HybridDataPuller:
    def __init__(self, fmp_api_key, tickers):
        self.fmp_api_key = fmp_api_key
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.fmp_base_url = "https://financialmodelingprep.com/stable"
        
    def _make_fmp_request(self, endpoint, params=None):
        """Make FMP API request with error handling"""
        if params is None:
            params = {}
        params['apikey'] = self.fmp_api_key
        
        try:
            response = requests.get(f"{self.fmp_base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"FMP API request failed: {e}")
            return None
    
    def get_current_prices(self):
        """Get current prices from Yahoo Finance (fast and free)"""
        print("Fetching current prices from Yahoo Finance...")
        price_data = {}
        
        for ticker in self.tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="1d")
                
                if not hist.empty:
                    price_data[ticker] = {
                        'current_price': hist['Close'].iloc[-1],
                        'timestamp': datetime.now()
                    }
                else:
                    print(f"Warning: No price data for {ticker}")
                    
            except Exception as e:
                print(f"Error fetching price data for {ticker}: {e}")
                
            time.sleep(0.1)  # Small delay
            
        return pd.DataFrame(price_data).T
    
    def get_fundamentals(self):
        """Get fundamental data from FMP (comprehensive and reliable)"""
        print("Fetching fundamental data from FMP...")
        fundamental_data = {}
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                # Get company profile
                profile_data = self._make_fmp_request(f"profile/{ticker}")
                
                # Get latest income statement (annual)
                income_data = self._make_fmp_request(f"income-statement/{ticker}", params={'limit': 1})
                
                # Get latest balance sheet (annual)  
                balance_data = self._make_fmp_request(f"balance-sheet-statement/{ticker}", params={'limit': 1})
                
                # Get latest cash flow (annual)
                cashflow_data = self._make_fmp_request(f"cash-flow-statement/{ticker}", params={'limit': 1})
                
                # Get ratios for PE ratio
                ratios_data = self._make_fmp_request(f"ratios-ttm/{ticker}")
                
                # Extract data (handle empty responses)
                profile = profile_data[0] if profile_data else {}
                income = income_data[0] if income_data else {}
                balance = balance_data[0] if balance_data else {}
                cashflow = cashflow_data[0] if cashflow_data else {}
                ratios = ratios_data[0] if ratios_data else {}
                
                fundamental_data[ticker] = {
                    'total_revenue': income.get('revenue'),
                    'operating_income': income.get('operatingIncome'),
                    'ebitda': income.get('ebitda'),
                    'net_income': income.get('netIncome'),
                    'total_debt': balance.get('totalDebt'),
                    'cash_and_equivalents': balance.get('cashAndCashEquivalents'),
                    'free_cash_flow': cashflow.get('freeCashFlow'),
                    'pe_ratio': ratios.get('priceEarningsRatio'),
                    'sector': profile.get('sector'),
                    'industry': profile.get('industry'),
                    'market_cap': profile.get('mktCap'),
                    'employee_count': profile.get('fullTimeEmployees'),
                    'last_updated': datetime.now()
                }
                
            except Exception as e:
                print(f"  Error fetching fundamentals for {ticker}: {e}")
                fundamental_data[ticker] = {
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.25)  # Rate limiting for FMP
            
        return pd.DataFrame(fundamental_data).T
    
    def get_quarterly_revenue(self, periods=8):
        """Get up to 8 quarters of revenue data from FMP (handling 5-record limit)"""
        print(f"Fetching up to {periods} quarters of revenue data from FMP...")
        quarterly_data = {}
        
        # Adjust periods to max 5 for free tier
        actual_periods = min(periods, 5)
        if periods > 5:
            print(f"  Note: Free tier limited to 5 quarters per call, getting {actual_periods} quarters")
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                # Get quarterly income statements using stable API format
                quarterly_income = self._make_fmp_request(
                    "income-statement", 
                    params={'symbol': ticker, 'period': 'quarter', 'limit': actual_periods}
                )
                
                if quarterly_income:
                    quarterly_revenue = []
                    
                    # Process quarterly data
                    for quarter in quarterly_income:
                        if quarter.get('revenue') is not None:
                            # Parse date and create quarter string
                            date_str = quarter.get('date', '')
                            if date_str:
                                try:
                                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                                    quarter_num = ((date_obj.month - 1) // 3) + 1
                                    quarter_str = f"{date_obj.year}-Q{quarter_num}"
                                except:
                                    quarter_str = date_str
                            else:
                                quarter_str = f"{quarter.get('calendarYear', 'Unknown')}"
                            
                            quarterly_revenue.append({
                                'quarter': quarter_str,
                                'date': date_str,
                                'revenue': quarter.get('revenue')
                            })
                    
                    quarterly_data[ticker] = {
                        'quarterly_revenue': quarterly_revenue,
                        'quarters_available': len(quarterly_revenue),
                        'last_updated': datetime.now()
                    }
                    
                    print(f"  Found {len(quarterly_revenue)} quarters")
                    
                else:
                    quarterly_data[ticker] = {
                        'quarterly_revenue': [],
                        'error': 'No quarterly data available',
                        'last_updated': datetime.now()
                    }
                    
            except Exception as e:
                print(f"  Error: {e}")
                quarterly_data[ticker] = {
                    'quarterly_revenue': [],
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.3)  # Rate limiting
            
        return quarterly_data
    
    def test_api_connection(self, ticker="AAPL"):
        """Test API connection with a single ticker to avoid wasting quota"""
        print(f"Testing API connection with {ticker}...")
        
        try:
            # Test the exact URL format you provided
            test_data = self._make_fmp_request(
                "income-statement", 
                params={'symbol': ticker}
            )
            
            print(f"Raw API response type: {type(test_data)}")
            print(f"Raw API response: {test_data}")
            
            if test_data:
                print(f"✅ API connection successful!")
                print(f"   Retrieved {len(test_data)} records")
                if len(test_data) > 0:
                    print(f"   First record keys: {list(test_data[0].keys())}")
                    print(f"   First record sample data:")
                    for key, value in list(test_data[0].items())[:10]:  # Show first 10 fields
                        print(f"     {key}: {value}")
                    
                    if 'revenue' in test_data[0]:
                        revenue = test_data[0]['revenue']
                        print(f"   Latest revenue: {revenue} (${revenue/1e9:.2f}B if numeric)")
                return True
            else:
                print("❌ API call failed - no data returned")
                return False
                
        except Exception as e:
            print(f"❌ API test failed: {e}")
            return False
    
    def get_all_data(self):
        """Get all data: current prices + fundamentals + quarterly revenue"""
        print(f"Fetching complete data for {len(self.tickers)} tickers...")
        
        results = {
            'current_prices': self.get_current_prices(),
            'fundamental_data': self.get_fundamentals(),
            'quarterly_revenue': self.get_quarterly_revenue(periods=8),
            'timestamp': datetime.now()
        }
        
        return results

# Usage example
if __name__ == "__main__":
    # Step 1: Get your free API key from https://financialmodelingprep.com/
    # API_KEY = "your_fmp_api_key_here"
    
    # Step 2: Define your tickers
    your_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'CRM', 'ORCL', 
                    'ADBE', 'PYPL', 'INTC', 'AMD', 'QCOM', 'TXN', 'AVGO', 'MU', 'AMAT', 'LRCX']
    
    # Step 3: Initialize and use
    # puller = HybridDataPuller(API_KEY, your_tickers)
    
    # Get current prices only (fast, for daily updates)
    # prices = puller.get_current_prices()
    
    # Get fundamentals only (for weekly/monthly updates)
    # fundamentals = puller.get_fundamentals()
    
    # Get 8 quarters of revenue data
    # quarterly_revenue = puller.get_quarterly_revenue(periods=8)
    
    # Get everything
    # all_data = puller.get_all_data()
    
    print("Hybrid Data Puller ready!")
    print("1. Sign up for free FMP API key: https://financialmodelingprep.com/")
    print("2. Replace 'your_fmp_api_key_here' with your actual API key")
    print("3. Uncomment the example code to test")

Hybrid Data Puller ready!
1. Sign up for free FMP API key: https://financialmodelingprep.com/
2. Replace 'your_fmp_api_key_here' with your actual API key
3. Uncomment the example code to test


In [9]:
# Test the API and see the raw results
API_KEY = "YUvsuP4XeCmHoyAioO5l4htdnL969zb7"
test_puller = HybridDataPuller(API_KEY, ['AAPL'])

# This will show us the complete raw response
success = test_puller.test_api_connection()

Testing API connection with AAPL...
Raw API response type: <class 'list'>
Raw API response: [{'date': '2024-09-28', 'symbol': 'AAPL', 'reportedCurrency': 'USD', 'cik': '0000320193', 'filingDate': '2024-11-01', 'acceptedDate': '2024-11-01 06:01:36', 'fiscalYear': '2024', 'period': 'FY', 'revenue': 391035000000, 'costOfRevenue': 210352000000, 'grossProfit': 180683000000, 'researchAndDevelopmentExpenses': 31370000000, 'generalAndAdministrativeExpenses': 0, 'sellingAndMarketingExpenses': 0, 'sellingGeneralAndAdministrativeExpenses': 26097000000, 'otherExpenses': 0, 'operatingExpenses': 57467000000, 'costAndExpenses': 267819000000, 'netInterestIncome': 0, 'interestIncome': 0, 'interestExpense': 0, 'depreciationAndAmortization': 11445000000, 'ebitda': 134661000000, 'ebit': 123216000000, 'nonOperatingIncomeExcludingInterest': 0, 'operatingIncome': 123216000000, 'totalOtherIncomeExpensesNet': 269000000, 'incomeBeforeTax': 123485000000, 'incomeTaxExpense': 29749000000, 'netIncomeFromContinuingO

In [14]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

class HybridDataPuller:
    def __init__(self, fmp_api_key, tickers):
        self.fmp_api_key = fmp_api_key
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.fmp_base_url = "https://financialmodelingprep.com/stable"
        
    def _make_fmp_request(self, endpoint, params=None):
        """Make FMP API request with error handling"""
        if params is None:
            params = {}
        params['apikey'] = self.fmp_api_key
        
        try:
            response = requests.get(f"{self.fmp_base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"FMP API request failed: {e}")
            return None
    
    def get_current_prices(self):
        """Get current prices from Yahoo Finance (fast and free)"""
        print("Fetching current prices from Yahoo Finance...")
        price_data = {}
        
        for ticker in self.tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="1d")
                
                if not hist.empty:
                    price_data[ticker] = {
                        'current_price': hist['Close'].iloc[-1],
                        'timestamp': datetime.now()
                    }
                else:
                    print(f"Warning: No price data for {ticker}")
                    
            except Exception as e:
                print(f"Error fetching price data for {ticker}: {e}")
                
            time.sleep(0.1)  # Small delay
            
        return pd.DataFrame(price_data).T
    
    def test_api_connection(self, ticker="AAPL"):
        """Test API connection with a single ticker to avoid wasting quota"""
        print(f"Testing API connection with {ticker}...")
        
        try:
            # Test the exact URL format you provided
            test_data = self._make_fmp_request(
                "income-statement", 
                params={'symbol': ticker}
            )
            
            print(f"Raw API response type: {type(test_data)}")
            print(f"Raw API response: {test_data}")
            
            if test_data:
                print(f"✅ API connection successful!")
                print(f"   Retrieved {len(test_data)} records")
                if len(test_data) > 0:
                    print(f"   First record keys: {list(test_data[0].keys())}")
                    print(f"   First record sample data:")
                    for key, value in list(test_data[0].items())[:10]:  # Show first 10 fields
                        print(f"     {key}: {value}")
                    
                    if 'revenue' in test_data[0]:
                        revenue = test_data[0]['revenue']
                        print(f"   Latest revenue: {revenue} (${revenue/1e9:.2f}B if numeric)")
                return True
            else:
                print("❌ API call failed - no data returned")
                return False
                
        except Exception as e:
            print(f"❌ API test failed: {e}")
            return False
    
    def test_quarterly_api(self, ticker="AAPL"):
        """Test quarterly API call specifically"""
        print(f"Testing QUARTERLY API connection with {ticker}...")
        
        try:
            # Test quarterly data specifically
            test_data = self._make_fmp_request(
                "income-statement", 
                params={'symbol': ticker, 'period': 'quarter', 'limit': 5}
            )
            
            print(f"Raw quarterly API response type: {type(test_data)}")
            print(f"Number of records: {len(test_data) if test_data else 0}")
            
            if test_data and len(test_data) > 0:
                print(f"✅ Quarterly API connection successful!")
                print(f"   Retrieved {len(test_data)} quarterly records")
                
                # Show first 3 quarterly records
                for i, record in enumerate(test_data[:3]):
                    print(f"\n   Quarter {i+1}:")
                    print(f"     Date: {record.get('date')}")
                    print(f"     Period: {record.get('period')}")
                    print(f"     Fiscal Year: {record.get('fiscalYear')}")
                    print(f"     Revenue: ${record.get('revenue', 0)/1e9:.2f}B")
                    print(f"     Operating Income: ${record.get('operatingIncome', 0)/1e9:.2f}B")
                    print(f"     EBITDA: ${record.get('ebitda', 0)/1e9:.2f}B")
                    print(f"     Net Income: ${record.get('netIncome', 0)/1e9:.2f}B")
                
                return True
            else:
                print("❌ No quarterly data returned")
                return False
                
        except Exception as e:
            print(f"❌ Quarterly API test failed: {e}")
            return False

# Usage example
if __name__ == "__main__":
    # Test the API
    API_KEY = "your_fmp_api_key_here"
    test_puller = HybridDataPuller(API_KEY, ['AAPL'])
    
    # Test annual data
    # success = test_puller.test_api_connection()
    
    # Test quarterly data  
    # success = test_puller.test_quarterly_api()
    
    print("Hybrid Data Puller ready for testing!")

Hybrid Data Puller ready for testing!


In [15]:
# Test quarterly data specifically
API_KEY = "YUvsuP4XeCmHoyAioO5l4htdnL969zb7"
test_puller = HybridDataPuller(API_KEY, ['AAPL'])

# Test quarterly API call
success = test_puller.test_quarterly_api()

Testing QUARTERLY API connection with AAPL...
Raw quarterly API response type: <class 'list'>
Number of records: 5
✅ Quarterly API connection successful!
   Retrieved 5 quarterly records

   Quarter 1:
     Date: 2025-06-28
     Period: Q3
     Fiscal Year: 2025
     Revenue: $94.04B
     Operating Income: $28.20B
     EBITDA: $31.03B
     Net Income: $23.43B

   Quarter 2:
     Date: 2025-03-29
     Period: Q2
     Fiscal Year: 2025
     Revenue: $95.36B
     Operating Income: $29.59B
     EBITDA: $32.25B
     Net Income: $24.78B

   Quarter 3:
     Date: 2024-12-28
     Period: Q1
     Fiscal Year: 2025
     Revenue: $124.30B
     Operating Income: $42.83B
     EBITDA: $45.91B
     Net Income: $36.33B


In [18]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

class HybridDataPuller:
    def __init__(self, fmp_api_key, tickers):
        self.fmp_api_key = fmp_api_key
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.fmp_base_url = "https://financialmodelingprep.com/stable"
        
    def _make_fmp_request(self, endpoint, params=None):
        """Make FMP API request with error handling"""
        if params is None:
            params = {}
        params['apikey'] = self.fmp_api_key
        
        try:
            response = requests.get(f"{self.fmp_base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"FMP API request failed: {e}")
            return None
    
    def get_current_prices(self):
        """Get current prices from Yahoo Finance (fast and free)"""
        print("Fetching current prices from Yahoo Finance...")
        price_data = {}
        
        for ticker in self.tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="1d")
                
                if not hist.empty:
                    price_data[ticker] = {
                        'current_price': hist['Close'].iloc[-1],
                        'timestamp': datetime.now()
                    }
                else:
                    print(f"Warning: No price data for {ticker}")
                    
            except Exception as e:
                print(f"Error fetching price data for {ticker}: {e}")
                
            time.sleep(0.1)  # Small delay
            
        return pd.DataFrame(price_data).T
    
    def test_period_q4(self, ticker="AAPL"):
        """Test what happens when we specify period='Q4'"""
        print(f"Testing period='Q4' with {ticker}...")
        
        try:
            # Test with period='Q4' specifically
            test_data = self._make_fmp_request(
                "income-statement", 
                params={'symbol': ticker, 'period': 'Q4', 'limit': 5}
            )
            
            print(f"Raw API response type: {type(test_data)}")
            print(f"Number of records: {len(test_data) if test_data else 0}")
            
            if test_data and len(test_data) > 0:
                print(f"✅ Q4 period query successful!")
                print(f"   Retrieved {len(test_data)} records")
                
                # Show all records
                for i, record in enumerate(test_data):
                    print(f"\n   Record {i+1}:")
                    print(f"     Date: {record.get('date')}")
                    print(f"     Period: {record.get('period')}")
                    print(f"     Fiscal Year: {record.get('fiscalYear')}")
                    print(f"     Revenue: ${record.get('revenue', 0)/1e9:.2f}B")
                
                return True
            else:
                print("❌ No data returned for Q4 period")
                return False
                
        except Exception as e:
            print(f"❌ Q4 period test failed: {e}")
            return False

    def get_quarterly_revenue(self, periods=10):
        """Get quarterly revenue data from FMP (trying 2 calls per ticker for up to 10 quarters)"""
        print(f"Fetching up to {periods} quarters of revenue data from FMP (2 calls per ticker)...")
        quarterly_data = {}
        
        for i, ticker in enumerate(self.tickers):
            print(f"Processing {ticker} ({i+1}/{len(self.tickers)})")
            
            try:
                quarterly_revenue = []
                
                # First call: Get the most recent 5 quarters
                print(f"  Making first API call for {ticker}...")
                quarterly_income_1 = self._make_fmp_request(
                    "income-statement", 
                    params={'symbol': ticker, 'period': 'quarter', 'limit': 5}
                )
                
                if quarterly_income_1:
                    quarterly_revenue.extend(quarterly_income_1)
                    print(f"  First call: Got {len(quarterly_income_1)} quarters")
                
                # Second call: Try to get older quarters 
                time.sleep(0.3)  # Small delay between calls for same ticker
                
                if len(quarterly_revenue) >= 5 and periods > 5:
                    print(f"  Making second API call for {ticker} (attempting older quarters)...")
                    
                    # Try getting more data by requesting a higher limit
                    quarterly_income_2 = self._make_fmp_request(
                        "income-statement", 
                        params={'symbol': ticker, 'period': 'quarter', 'limit': 10}
                    )
                    
                    if quarterly_income_2 and len(quarterly_income_2) > len(quarterly_revenue):
                        # If we got more data, replace our results
                        quarterly_revenue = quarterly_income_2
                        print(f"  Second call: Got {len(quarterly_income_2)} total quarters")
                    else:
                        print(f"  Second call: No additional quarters found")
                
                # Process all collected quarterly data
                processed_quarters = []
                for quarter in quarterly_revenue[:periods]:  # Limit to requested periods
                    if quarter.get('revenue') is not None:
                        # Create proper quarter string from the data
                        date_str = quarter.get('date', '')
                        period = quarter.get('period', '')
                        fiscal_year = quarter.get('fiscalYear', '')
                        
                        # Use the fiscal year and quarter from the API response
                        quarter_str = f"{fiscal_year}-{period}" if fiscal_year and period else date_str
                        
                        processed_quarters.append({
                            'quarter': quarter_str,
                            'date': date_str,
                            'fiscal_year': fiscal_year,
                            'period': period,
                            'revenue': quarter.get('revenue')
                        })
                
                quarterly_data[ticker] = {
                    'quarterly_revenue': processed_quarters,
                    'quarters_available': len(processed_quarters),
                    'api_calls_made': 2 if periods > 5 else 1,
                    'last_updated': datetime.now()
                }
                
                print(f"  Final result: {len(processed_quarters)} quarters for {ticker}")
                    
            except Exception as e:
                print(f"  Error: {e}")
                quarterly_data[ticker] = {
                    'quarterly_revenue': [],
                    'error': str(e),
                    'last_updated': datetime.now()
                }
                
            time.sleep(0.5)  # Longer delay due to multiple calls per ticker
            
        return quarterly_data

# Usage example
if __name__ == "__main__":
    # Test the API
    API_KEY = "your_fmp_api_key_here"
    test_puller = HybridDataPuller(API_KEY, ['AAPL'])
    
    # Test Q4 period specifically
    # success = test_puller.test_period_q4()
    
    print("Hybrid Data Puller ready for testing!")

Hybrid Data Puller ready for testing!


In [19]:
# Test Q4 period specifically
API_KEY = "YUvsuP4XeCmHoyAioO5l4htdnL969zb7"
test_puller = HybridDataPuller(API_KEY, ['AAPL'])

# Test with period='Q4'
test_puller.test_period_q4()

Testing period='Q4' with AAPL...
Raw API response type: <class 'list'>
Number of records: 5
✅ Q4 period query successful!
   Retrieved 5 records

   Record 1:
     Date: 2024-09-28
     Period: Q4
     Fiscal Year: 2024
     Revenue: $94.93B

   Record 2:
     Date: 2023-09-30
     Period: Q4
     Fiscal Year: 2023
     Revenue: $89.50B

   Record 3:
     Date: 2022-09-24
     Period: Q4
     Fiscal Year: 2022
     Revenue: $90.15B

   Record 4:
     Date: 2021-09-25
     Period: Q4
     Fiscal Year: 2021
     Revenue: $83.36B

   Record 5:
     Date: 2020-09-26
     Period: Q4
     Fiscal Year: 2020
     Revenue: $64.70B


True

In [25]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime, timedelta
import time

class StockDataManager:
    def __init__(self, fmp_api_key, data_folder="stock_data"):
        self.fmp_api_key = fmp_api_key
        self.fmp_base_url = "https://financialmodelingprep.com/stable"
        self.data_folder = data_folder
        
        # Create data folder if it doesn't exist
        os.makedirs(data_folder, exist_ok=True)
        print(f"Data will be saved to: {os.path.abspath(data_folder)}")
        
    def _make_fmp_request(self, endpoint, params=None):
        """Make FMP API request with error handling"""
        if params is None:
            params = {}
        params['apikey'] = self.fmp_api_key
        
        try:
            response = requests.get(f"{self.fmp_base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"FMP API request failed: {e}")
            return None
    
    def _save_ticker_data(self, ticker, data):
        """Save ticker data to local JSON file"""
        filename = f"{self.data_folder}/{ticker}_quarterly_data.json"
        try:
            with open(filename, 'w') as f:
                json.dump(data, f, indent=2, default=str)
            print(f"✅ Saved data for {ticker} to {filename}")
        except Exception as e:
            print(f"❌ Error saving data for {ticker}: {e}")
    
    def _load_ticker_data(self, ticker):
        """Load ticker data from local JSON file"""
        filename = f"{self.data_folder}/{ticker}_quarterly_data.json"
        try:
            if os.path.exists(filename):
                with open(filename, 'r') as f:
                    return json.load(f)
            else:
                print(f"No existing data found for {ticker}")
                return None
        except Exception as e:
            print(f"❌ Error loading data for {ticker}: {e}")
            return None
    
    def initialize_ticker_historical_data(self, ticker):
        """Initialize a new ticker with 5 years of quarterly data (Q1, Q2, Q3, Q4)"""
        print(f"\n🚀 INITIALIZING {ticker} - Collecting 5 years of quarterly data...")
        print("This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history")
        
        all_quarters = []
        api_calls_made = 0
        
        # Pull each quarter separately to get maximum historical data
        for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
            print(f"\n📊 Fetching {quarter} data for {ticker}...")
            
            try:
                quarterly_data = self._make_fmp_request(
                    "income-statement",
                    params={'symbol': ticker, 'period': quarter, 'limit': 5}
                )
                
                api_calls_made += 1
                
                if quarterly_data:
                    print(f"  ✅ Got {len(quarterly_data)} {quarter} quarters")
                    
                    # Process each quarter - SAVE ALL AVAILABLE METRICS
                    for quarter_record in quarterly_data:
                        processed_quarter = {
                            # Add quarter identification
                            'quarter_label': f"{quarter_record.get('fiscalYear')}-{quarter_record.get('period')}",
                            
                            # Save ALL metrics from the API response
                            **quarter_record  # This includes all available fields
                        }
                        all_quarters.append(processed_quarter)
                        
                        # Show progress
                        rev = quarter_record.get('revenue', 0)
                        print(f"    {processed_quarter['quarter_label']}: ${rev/1e9:.2f}B revenue")
                else:
                    print(f"  ❌ No {quarter} data returned")
                    
            except Exception as e:
                print(f"  ❌ Error fetching {quarter} data: {e}")
            
            time.sleep(0.3)  # Rate limiting between calls
        
        # Sort quarters by date (most recent first)
        all_quarters.sort(key=lambda x: x.get('date', '') if x.get('date') else '', reverse=True)
        
        # Create comprehensive ticker data structure
        ticker_data = {
            'ticker': ticker,
            'initialization_date': datetime.now().isoformat(),
            'total_quarters': len(all_quarters),
            'api_calls_made': api_calls_made,
            'quarterly_data': all_quarters,
            'data_summary': {
                'earliest_quarter': all_quarters[-1]['quarter_label'] if all_quarters else None,
                'latest_quarter': all_quarters[0]['quarter_label'] if all_quarters else None,
                'years_covered': len(set(q['fiscalYear'] for q in all_quarters if q.get('fiscalYear'))),
            }
        }
        
        # Save to local file
        self._save_ticker_data(ticker, ticker_data)
        
        # Print summary
        print(f"\n🎉 INITIALIZATION COMPLETE for {ticker}")
        print(f"   📈 Total quarters collected: {len(all_quarters)}")
        print(f"   📅 Date range: {ticker_data['data_summary']['earliest_quarter']} to {ticker_data['data_summary']['latest_quarter']}")
        print(f"   🗓️  Years covered: {ticker_data['data_summary']['years_covered']}")
        print(f"   🔢 API calls used: {api_calls_made}")
        
        return ticker_data
    
    def update_ticker_latest_data(self, ticker):
        """Update ticker with latest quarterly data"""
        print(f"\n🔄 UPDATING {ticker} with latest quarterly data...")
        
        # Load existing data
        existing_data = self._load_ticker_data(ticker)
        if not existing_data:
            print(f"❌ No existing data for {ticker}. Run initialize_ticker_historical_data() first.")
            return None
        
        try:
            # Get latest quarterly data
            latest_data = self._make_fmp_request(
                "income-statement",
                params={'symbol': ticker, 'period': 'quarter', 'limit': 3}
            )
            
            if latest_data:
                print(f"  ✅ Retrieved {len(latest_data)} latest quarters")
                
                # Check for new quarters
                existing_quarters = set(q['quarter_label'] for q in existing_data['quarterly_data'])
                new_quarters_added = 0
                
                for quarter_record in latest_data:
                    quarter_label = f"{quarter_record.get('fiscalYear')}-{quarter_record.get('period')}"
                    
                    if quarter_label not in existing_quarters:
                        # Add new quarter - SAVE ALL AVAILABLE METRICS
                        processed_quarter = {
                            'quarter_label': quarter_label,
                            **quarter_record  # Save all metrics from API response
                        }
                        
                        existing_data['quarterly_data'].insert(0, processed_quarter)  # Add to beginning
                        new_quarters_added += 1
                        
                        rev = quarter_record.get('revenue', 0)
                        print(f"    ➕ Added {quarter_label}: ${rev/1e9:.2f}B revenue")
                
                if new_quarters_added > 0:
                    # Re-sort quarterly data by date (most recent first)
                    existing_data['quarterly_data'].sort(key=lambda x: x.get('date', '') if x.get('date') else '', reverse=True)
                    
                    # Update metadata
                    existing_data['last_update'] = datetime.now().isoformat()
                    existing_data['total_quarters'] = len(existing_data['quarterly_data'])
                    existing_data['data_summary']['latest_quarter'] = existing_data['quarterly_data'][0]['quarter_label']
                    
                    # Save updated data
                    self._save_ticker_data(ticker, existing_data)
                    print(f"  🎉 Added {new_quarters_added} new quarters for {ticker}")
                else:
                    print(f"  ℹ️  No new quarters found for {ticker}")
                    
                return existing_data
                
        except Exception as e:
            print(f"❌ Error updating {ticker}: {e}")
            return None
    
    def show_available_metrics(self, ticker):
        """Show all available metrics for a ticker"""
        data = self._load_ticker_data(ticker)
        if data and data['quarterly_data']:
            sample_quarter = data['quarterly_data'][0]
            print(f"\n📊 Available metrics for {ticker}:")
            print("=" * 50)
            
            # Group metrics by category for better readability
            financial_metrics = ['revenue', 'costOfRevenue', 'grossProfit', 'operatingIncome', 'ebitda', 'netIncome']
            expense_metrics = ['researchAndDevelopmentExpenses', 'sellingGeneralAndAdministrativeExpenses', 'operatingExpenses']
            per_share_metrics = ['eps', 'epsDiluted', 'weightedAverageShsOut', 'weightedAverageShsOutDil']
            
            print("💰 Key Financial Metrics:")
            for metric in financial_metrics:
                if metric in sample_quarter:
                    value = sample_quarter[metric]
                    if isinstance(value, (int, float)) and value:
                        print(f"  {metric}: ${value/1e9:.2f}B")
                    else:
                        print(f"  {metric}: {value}")
            
            print("\n💸 Expense Metrics:")
            for metric in expense_metrics:
                if metric in sample_quarter:
                    value = sample_quarter[metric]
                    if isinstance(value, (int, float)) and value:
                        print(f"  {metric}: ${value/1e9:.2f}B")
            
            print("\n📈 Per Share Metrics:")
            for metric in per_share_metrics:
                if metric in sample_quarter:
                    print(f"  {metric}: {sample_quarter[metric]}")
            
            print("\n🔍 All Available Fields:")
            all_fields = list(sample_quarter.keys())
            print(f"  Total fields: {len(all_fields)}")
            print(f"  Fields: {', '.join(all_fields)}")
        else:
            print(f"❌ No data found for {ticker}")
    
    def get_ticker_data(self, ticker):
        """Get stored ticker data"""
        return self._load_ticker_data(ticker)
    
    def get_current_price(self, ticker):
        """Get current price from Yahoo Finance"""
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1d")
            if not hist.empty:
                return hist['Close'].iloc[-1]
        except Exception as e:
            print(f"Error getting price for {ticker}: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Initialize the manager
    API_KEY = "your_fmp_api_key_here"
    manager = StockDataManager(API_KEY)
    
    # Initialize AAPL with historical data
    # aapl_data = manager.initialize_ticker_historical_data("AAPL")
    
    # Later: Update with latest data
    # updated_data = manager.update_ticker_latest_data("AAPL") 
    
    # Get stored data
    # stored_data = manager.get_ticker_data("AAPL")
    
    print("Stock Data Manager ready!")
    print("Use: manager.initialize_ticker_historical_data('AAPL') to start")

Data will be saved to: /Users/yvonneyhz/stock_data
Stock Data Manager ready!
Use: manager.initialize_ticker_historical_data('AAPL') to start


In [22]:
# Initialize the manager with your specific path
API_KEY = "YUvsuP4XeCmHoyAioO5l4htdnL969zb7"
data_path = "/Users/yvonneyhz/Dropbox/2025 Coding/yz-website/stock_data/"
manager = StockDataManager(API_KEY, data_folder=data_path)

# This will save data to your Dropbox folder
aapl_data = manager.initialize_ticker_historical_data("AAPL")

Data will be saved to: /Users/yvonneyhz/Dropbox/2025 Coding/yz-website/stock_data

🚀 INITIALIZING AAPL - Collecting 5 years of quarterly data...
This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history

📊 Fetching Q1 data for AAPL...
  ✅ Got 5 Q1 quarters
    2025-Q1: $124.30B revenue
    2024-Q1: $119.58B revenue
    2023-Q1: $117.15B revenue
    2022-Q1: $123.94B revenue
    2021-Q1: $111.44B revenue

📊 Fetching Q2 data for AAPL...
  ✅ Got 5 Q2 quarters
    2025-Q2: $95.36B revenue
    2024-Q2: $90.75B revenue
    2023-Q2: $94.84B revenue
    2022-Q2: $97.28B revenue
    2021-Q2: $89.58B revenue

📊 Fetching Q3 data for AAPL...
  ✅ Got 5 Q3 quarters
    2025-Q3: $94.04B revenue
    2024-Q3: $85.78B revenue
    2023-Q3: $81.80B revenue
    2022-Q3: $82.96B revenue
    2021-Q3: $81.43B revenue

📊 Fetching Q4 data for AAPL...
  ✅ Got 5 Q4 quarters
    2024-Q4: $94.93B revenue
    2023-Q4: $89.50B revenue
    2022-Q4: $90.15B revenue
    2021-Q4: $83.36B revenue
    2020

In [26]:
# Initialize with your path and get ALL metrics
API_KEY = "YUvsuP4XeCmHoyAioO5l4htdnL969zb7"
data_path = "/Users/yvonneyhz/Dropbox/2025 Coding/yz-website/stock_data/"
manager = StockDataManager(API_KEY, data_folder=data_path)

# This should work now
aapl_data = manager.initialize_ticker_historical_data("AAPL")

# After initialization, see what metrics you have:
manager.show_available_metrics("AAPL")

Data will be saved to: /Users/yvonneyhz/Dropbox/2025 Coding/yz-website/stock_data

🚀 INITIALIZING AAPL - Collecting 5 years of quarterly data...
This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history

📊 Fetching Q1 data for AAPL...
  ✅ Got 5 Q1 quarters
    2025-Q1: $124.30B revenue
    2024-Q1: $119.58B revenue
    2023-Q1: $117.15B revenue
    2022-Q1: $123.94B revenue
    2021-Q1: $111.44B revenue

📊 Fetching Q2 data for AAPL...
  ✅ Got 5 Q2 quarters
    2025-Q2: $95.36B revenue
    2024-Q2: $90.75B revenue
    2023-Q2: $94.84B revenue
    2022-Q2: $97.28B revenue
    2021-Q2: $89.58B revenue

📊 Fetching Q3 data for AAPL...
  ✅ Got 5 Q3 quarters
    2025-Q3: $94.04B revenue
    2024-Q3: $85.78B revenue
    2023-Q3: $81.80B revenue
    2022-Q3: $82.96B revenue
    2021-Q3: $81.43B revenue

📊 Fetching Q4 data for AAPL...
  ✅ Got 5 Q4 quarters
    2024-Q4: $94.93B revenue
    2023-Q4: $89.50B revenue
    2022-Q4: $90.15B revenue
    2021-Q4: $83.36B revenue
    2020

In [27]:
# BATCH 1 - Big Tech + EV
batch_1_tickers = ["TSLA", "AMZN", "MSFT", "NVDA", "GOOGL", "META"]

print("🚀 Starting BATCH 1 initialization...")
print(f"Tickers: {batch_1_tickers}")
print("This will use approximately 24 API calls (4 per ticker)")

batch_1_results = {}
for ticker in batch_1_tickers:
    try:
        print(f"\n" + "="*60)
        result = manager.initialize_ticker_historical_data(ticker)
        batch_1_results[ticker] = "✅ Success"
        print(f"✅ {ticker} completed successfully")
    except Exception as e:
        batch_1_results[ticker] = f"❌ Error: {e}"
        print(f"❌ {ticker} failed: {e}")
        
print(f"\n🏁 BATCH 1 COMPLETE!")
print("Results:")
for ticker, status in batch_1_results.items():
    print(f"  {ticker}: {status}")

🚀 Starting BATCH 1 initialization...
Tickers: ['TSLA', 'AMZN', 'MSFT', 'NVDA', 'GOOGL', 'META']
This will use approximately 24 API calls (4 per ticker)


🚀 INITIALIZING TSLA - Collecting 5 years of quarterly data...
This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history

📊 Fetching Q1 data for TSLA...
  ✅ Got 5 Q1 quarters
    2025-Q1: $19.34B revenue
    2024-Q1: $21.30B revenue
    2023-Q1: $23.33B revenue
    2022-Q1: $18.76B revenue
    2021-Q1: $10.39B revenue

📊 Fetching Q2 data for TSLA...
  ✅ Got 5 Q2 quarters
    2025-Q2: $22.50B revenue
    2024-Q2: $25.50B revenue
    2023-Q2: $24.93B revenue
    2022-Q2: $16.93B revenue
    2021-Q2: $11.96B revenue

📊 Fetching Q3 data for TSLA...
  ✅ Got 5 Q3 quarters
    2024-Q3: $25.18B revenue
    2023-Q3: $23.35B revenue
    2022-Q3: $21.45B revenue
    2021-Q3: $13.76B revenue
    2020-Q3: $8.77B revenue

📊 Fetching Q4 data for TSLA...
  ✅ Got 5 Q4 quarters
    2024-Q4: $25.71B revenue
    2023-Q4: $25.17B revenue
 

In [28]:
# BATCH 2 - Growth Tech + Emerging
batch_2_tickers = ["NFLX", "AMD", "PLTR", "SHOP", "SNAP", "UBER"]

print("🚀 Starting BATCH 2 initialization...")
print(f"Tickers: {batch_2_tickers}")
print("This will use approximately 24 API calls (4 per ticker)")

batch_2_results = {}
for ticker in batch_2_tickers:
    try:
        print(f"\n" + "="*60)
        result = manager.initialize_ticker_historical_data(ticker)
        batch_2_results[ticker] = "✅ Success"
        print(f"✅ {ticker} completed successfully")
    except Exception as e:
        batch_2_results[ticker] = f"❌ Error: {e}"
        print(f"❌ {ticker} failed: {e}")
        
print(f"\n🏁 BATCH 2 COMPLETE!")
print("Results:")
for ticker, status in batch_2_results.items():
    print(f"  {ticker}: {status}")

🚀 Starting BATCH 2 initialization...
Tickers: ['NFLX', 'AMD', 'PLTR', 'SHOP', 'SNAP', 'UBER']
This will use approximately 24 API calls (4 per ticker)


🚀 INITIALIZING NFLX - Collecting 5 years of quarterly data...
This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history

📊 Fetching Q1 data for NFLX...
  ✅ Got 5 Q1 quarters
    2025-Q1: $10.54B revenue
    2024-Q1: $9.38B revenue
    2023-Q1: $8.16B revenue
    2022-Q1: $7.87B revenue
    2021-Q1: $7.16B revenue

📊 Fetching Q2 data for NFLX...
  ✅ Got 5 Q2 quarters
    2025-Q2: $11.08B revenue
    2024-Q2: $9.56B revenue
    2023-Q2: $8.19B revenue
    2022-Q2: $7.97B revenue
    2021-Q2: $7.34B revenue

📊 Fetching Q3 data for NFLX...
  ✅ Got 5 Q3 quarters
    2024-Q3: $9.82B revenue
    2023-Q3: $8.54B revenue
    2022-Q3: $7.93B revenue
    2021-Q3: $7.48B revenue
    2020-Q3: $6.44B revenue

📊 Fetching Q4 data for NFLX...
  ✅ Got 5 Q4 quarters
    2024-Q4: $10.25B revenue
    2023-Q4: $8.83B revenue
    2022-Q4: $7.

In [29]:
# BATCH 3 - E-commerce + Fintech + Creative
batch_3_tickers = ["ETSY", "COIN", "PINS", "ADBE", "RKT"]

print("🚀 Starting BATCH 3 initialization...")
print(f"Tickers: {batch_3_tickers}")
print("This will use approximately 20 API calls (4 per ticker)")

batch_3_results = {}
for ticker in batch_3_tickers:
    try:
        print(f"\n" + "="*60)
        result = manager.initialize_ticker_historical_data(ticker)
        batch_3_results[ticker] = "✅ Success"
        print(f"✅ {ticker} completed successfully")
    except Exception as e:
        batch_3_results[ticker] = f"❌ Error: {e}"
        print(f"❌ {ticker} failed: {e}")
        
print(f"\n🏁 BATCH 3 COMPLETE!")
print("Results:")
for ticker, status in batch_3_results.items():
    print(f"  {ticker}: {status}")

# Final summary of all initialized tickers
print(f"\n🎉 ALL BATCHES COMPLETE!")
print("You now have historical data for:")
all_tickers = ["AAPL", "TSLA", "AMZN", "MSFT", "NVDA", "GOOGL", "META", "NFLX", "AMD", "PLTR", "SHOP", "SNAP", "UBER", "ETSY", "COIN", "PINS", "ADBE", "RKT"]
for i, ticker in enumerate(all_tickers, 1):
    print(f"  {i:2}. {ticker}")

print(f"\nTotal: {len(all_tickers)} companies with ~20 quarters each")
print(f"Data saved to: {data_path}")

🚀 Starting BATCH 3 initialization...
Tickers: ['ETSY', 'COIN', 'PINS', 'ADBE', 'RKT']
This will use approximately 20 API calls (4 per ticker)


🚀 INITIALIZING ETSY - Collecting 5 years of quarterly data...
This will make 4 API calls (Q1, Q2, Q3, Q4) to get ~20 quarters of history

📊 Fetching Q1 data for ETSY...
  ✅ Got 5 Q1 quarters
    2025-Q1: $0.65B revenue
    2024-Q1: $0.65B revenue
    2023-Q1: $0.64B revenue
    2022-Q1: $0.58B revenue
    2021-Q1: $0.55B revenue

📊 Fetching Q2 data for ETSY...
  ✅ Got 5 Q2 quarters
    2025-Q2: $0.67B revenue
    2024-Q2: $0.65B revenue
    2023-Q2: $0.63B revenue
    2022-Q2: $0.59B revenue
    2021-Q2: $0.53B revenue

📊 Fetching Q3 data for ETSY...
  ✅ Got 5 Q3 quarters
    2024-Q3: $0.66B revenue
    2023-Q3: $0.64B revenue
    2022-Q3: $0.59B revenue
    2021-Q3: $0.53B revenue
    2020-Q3: $0.45B revenue

📊 Fetching Q4 data for ETSY...
  ✅ Got 5 Q4 quarters
    2024-Q4: $0.85B revenue
    2023-Q4: $0.84B revenue
    2022-Q4: $0.81B revenue